#  Nique & Punch's Farm

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

# import library

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions


In [3]:
# Load the pre-trained MobileNetV2 model (trained on ImageNet)
model = MobileNetV2(weights='imagenet')

# Define a list of fruit and vegetable keywords (all lowercase)
fruit_veg_keywords = [
    'banana', 'apple', 'orange', 'strawberry', 'lemon',
    'pineapple', 'broccoli', 'cucumber', 'carrot', 'tomato', 'pepper'
]

def is_fruit_or_veg(predictions, keywords=fruit_veg_keywords):
    """
    Check if any of the predictions match the fruit/vegetable keywords.
    Returns a list of matching (class_name, score) tuples.
    """
    matches = []
    for pred in predictions[0]:
        class_name = pred[1].lower()
        score = pred[2]
        for kw in keywords:
            if kw in class_name:
                matches.append((class_name, score))
    return matches

# Initialize the webcam (device 0 is typically the default)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open the webcam.")
    exit()

# Set the grid spacing (in pixels) for the fine grid overlay
grid_spacing = 20

# Variable to hold overlay text (initially None)
overlay_text = None

print("Webcam is open.")
print("Press 'c' to capture an image and perform recognition.")
print("Press 'q' to quit.")

while True:
    # Capture the current frame from the webcam
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame from the webcam.")
        break

    height, width, _ = frame.shape

    # Draw fine grid lines across the image
    for x in range(0, width, grid_spacing):
        cv2.line(frame, (x, 0), (x, height), (0, 255, 0), 1)
    for y in range(0, height, grid_spacing):
        cv2.line(frame, (0, y), (width, y), (0, 255, 0), 1)

    # If overlay text is available, draw a small text box on the frame
    if overlay_text is not None:
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.7
        thickness = 2
        # Calculate text size for the overlay
        text_size, _ = cv2.getTextSize(overlay_text, font, font_scale, thickness)
        text_width, text_height = text_size
        x0, y0 = 10, 10  # top-left corner for the text box
        # Draw a filled rectangle as the background for the text
        cv2.rectangle(frame, (x0, y0), (x0 + text_width + 10, y0 + text_height + 10), (0, 0, 0), -1)
        # Put the overlay text over the rectangle in white
        cv2.putText(frame, overlay_text, (x0 + 5, y0 + text_height + 5), font, font_scale, (255, 255, 255), thickness)

    # Display the webcam feed with the grid and overlay text
    cv2.imshow("Webcam Feed", frame)

    # Check for key presses
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):
        # 'c' pressed: Capture the current frame for recognition
        captured_frame = frame.copy()
        # Resize and preprocess the image for MobileNetV2 (expected size: 224x224)
        img_resized = cv2.resize(captured_frame, (224, 224))
        x_input = np.expand_dims(img_resized, axis=0)
        x_input = preprocess_input(x_input)

        # Run the image through the model to get predictions
        preds = model.predict(x_input)
        decoded_preds = decode_predictions(preds, top=5)
        print("\nTop predictions:", decoded_preds)

        # Check if any of the predictions correspond to fruits or vegetables
        matches = is_fruit_or_veg(decoded_preds)
        if matches:
            best_match = max(matches, key=lambda x: x[1])
            overlay_text = f"It is {best_match[0].capitalize()} ({best_match[1]*100:.1f}%)"
            print("Detected fruit/vegetable:", overlay_text)
        else:
            overlay_text = "No fruit/vegetable detected"
            print("No fruit/vegetable detected.")

    elif key == ord('q'):
        # 'q' pressed: Exit the loop and close the program
        print("Exiting the program.")
        break

# Release the webcam and add a short delay before closing windows
cap.release()
cv2.waitKey(100)
cv2.destroyAllWindows()

Webcam is open.
Press 'c' to capture an image and perform recognition.
Press 'q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step

Top predictions: [[('n02782093', 'balloon', np.float32(0.24237348)), ('n03956157', 'planetarium', np.float32(0.107386544)), ('n02692877', 'airship', np.float32(0.100592405)), ('n03857828', 'oscilloscope', np.float32(0.07930007)), ('n03196217', 'digital_clock', np.float32(0.028226126))]]
No fruit/vegetable detected.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step

Top predictions: [[('n02782093', 'balloon', np.float32(0.1952009)), ('n03956157', 'planetarium', np.float32(0.10754419)), ('n04366367', 'suspension_bridge', np.float32(0.05575896)), ('n02692877', 'airship', np.float32(0.04779819)), ('n04540053', 'volleyball', np.float32(0.035780434))]]
No fruit/vegetable detected.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

Top predictions: [[('n03857828', 'oscilloscope', np.float32(0.59291846)), ('n03347037', 'fire_screen', np.float32(0.018615019)), ('n03929660', 'pick', np.float